# Modelling firing regularity in the ventral cochlear nucleus: mechanisms, and effects of stimulus level and synaptopathy

**Dan F. M. Goodman, Ian M. Winter, Agnès C. Léger, Alain de Cheveigné, Christian Lorenzi**

See the published version of the paper [here](https://doi.org/10.1016/j.heares.2017.09.010), and the freely accessible preprint [here](https://www.biorxiv.org/content/early/2017/09/19/121707).

The interactive paper has four notebooks, each providing interactive versions of some of the figures in the paper. The notebooks are listed below:

1. [Basic model](basic_model.ipynb)
2. [Behaviour maps](maps.ipynb)
3. [Level dependence](level_dependence_density.ipynb)
4. [Deafferentation](deafferentation.ipynb)